# COGS 108 - Final Project (change this to your project's title)

## Permissions

Place an `X` in the appropriate bracket below to specify if you would like your group's project to be made available to the public. (Note that student names will be included (but PIDs will be scraped from any groups who include their PIDs).

* [  ] YES - make available
* [  ] NO - keep private

# Overview

*Fill in your overview here*

# Names

- Nilou Shahbandi
- Leonardo Gonzalez
- Brian Chen
- Tin-Trung Pham
- Zhongkang(ken) Fang

<a id='research_question'></a>
# Research Question

*Fill in your research question here*

<a id='background'></a>

## Background & Prior Work

*Fill in your background and prior work here* 

References (include links):
- 1)
- 2)

# Hypothesis


*Fill in your hypotheses here*

# Dataset(s)

### *Dataset 1*

(Copy this information for each dataset)
- Dataset Name: annual_aqi_by_county_2010
- Link to the dataset: https://aqs.epa.gov/aqsweb/airdata/download_files.html
- Number of observations: 999


- data contains AQI, calculated each day for each monitor for the Criteria Gases and PM10 and PM2.5 (FRM and non FRM).
- of Days CO, NO2, Ozone, SO2, PM2.5, PM10 where the daily monitor criteria were met.

- We extract the year of 2010 from the cancer data and merge with this data.
- We will also add more years differentiate by colums and merge with cancer data which already have different years.


### *Dataset 2*

(Copy this information for each dataset)
- Dataset Name: Lung and Bronchus Cancer Incidence in California
- Link to the dataset:[California Cancer Registry](https://explorer.ccrcal.org/application.html?site=47&data_type=1&graph_type=13&compareBy=sex&chk_sex_1=1&race=1&age_range=9&hdn_stage=101&year=4&advopt_precision=4&advopt_display=2#tableWrap)
- Number of observations: 9306
- 2010 - 2018

This data shows us the counts of lung or bronchus cancer in individuals of all races and sexes by county in California. It also includes breakdowns by age with additional statistics regarding the data.

We plan to use this data in combination with the pollution data to track the impact of pollution on lung or bronchus cancer indcidents in individuals living in California. By comparing changes in pollution with changes in cancer incident counts.


### *Dataset 3*

(Copy this information for each dataset)
- Dataset Name: 
- Link to the dataset:
- Number of observations:

#### Description:
- 


#### If you plan to use multiple datasets, add 1-2 sentences about how you plan to combine these datasets.
- 

# Setup

In [7]:
## YOUR CODE HERE
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.style as style
import seaborn as sns

aqi_2010 = pd.read_csv('data/aqireport2010.csv')
county_cancer_all = pd.read_csv('data/countyCancer.csv')
asthma = pd.read_csv('data/Asthma.csv')

# Data Cleaning

Describe your data cleaning steps here.

In [8]:
## YOUR CODE HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION
asthma.head()

,State,ChildCount,Percent,Year,AdultCount,Percent.1
0,Alabama,"141,500",13.20%,2014.0,"394,600",10.50%
1,Alaska,---,---,NaN,"50,700",9.20%
2,Arizona,"129,200",8.10%,2016.0,"545,100",10.00%
3,Arkansas,---,---,NaN,"225,900",9.80%
4,California,"542,700",6.20%,NaN,"2,607,600",8.50%


In [9]:
aqi_2010.drop(['# Days with AQI', 'Moderate', 'Unhealthy', 'Very Unhealthy', 'County Code',
                'Hazardous', 'AQI Maximum', '# Days CO', '# Days NO2', '# Days O3', '# Days PM2.5', '# Days PM10'  ], 
                 inplace = True, axis=1)
aqi_2010.columns = ['county', 'Good days', 'Unhealthy days', 'AQI 90th Percentile', 'AQI Median']
aqi_2010['county'] = aqi_2010['county'].apply(standardize_region)
aqi_2010.head()

,county,Good days,Unhealthy days,AQI 90th Percentile,AQI Median
0,alameda,245,4,68.0,43.0
1,amador,302,10,64.0,35.0
2,butte,208,13,84.0,47.0
3,calaveras,271,12,84.0,41.0
4,colusa,322,1,49.0,38.0


In [10]:
county_cancer_all = pd.read_csv('data/countyCancer.csv')
county_cancer_all = county_cancer_all[county_cancer_all['Age'] == 'All Ages']
county_cancer_all.drop(['Sex', 'Race/Ethnicity', 'Age', 'Stage at Diagnosis', 'Age-adjusted Rate Standard Error',
                         'Age-adjusted Rate Lower Confidence Interval', 'Age-adjusted Rate Upper Confidence Interval'], 
                 inplace = True, axis=1)
county_cancer_all['California Region'] = county_cancer_all['California Region'].apply(standardize_region)
county_cancer_all.columns = ['year','county', 'rate per 100,000', 'count']
county_cancer_all = county_cancer_all[county_cancer_all['rate per 100,000'] != 'a']
county_cancer_all.head()

,year,county,"rate per 100,000",count
0,2010,alameda,44.8118,646
1,2010,butte,63.0616,171
2,2010,contra costa,50.2429,536
3,2010,el dorado,54.2364,124
4,2010,fresno,51.7782,401


In [11]:
county_cancer_2010 = county_cancer[county_cancer['Year'] == 2010]
county_cancer_2010 = county_cancer_2010[county_cancer_2010['Age'] == 'All Ages']
county_cancer_2010.drop(['Sex', 'Race/Ethnicity', 'Age', 'Stage at Diagnosis', 'Age-adjusted Rate Standard Error', 'Year',
                         'Age-adjusted Rate Lower Confidence Interval', 'Age-adjusted Rate Upper Confidence Interval'], 
                 inplace = True, axis=1)
county_cancer_2010['California Region'] = county_cancer_2010['California Region'].apply(standardize_region)
county_cancer_2010.columns = ['county', 'rate per 100,000', 'count']
county_cancer_2010.head()

,county,"rate per 100,000",count
0,alameda,44.8118,646
1,butte,63.0616,171
2,contra costa,50.2429,536
3,el dorado,54.2364,124
4,fresno,51.7782,401


In [5]:
def standardize_region(str_in):
    
    try:
        str_in = str_in.lower()
        str_in = str_in.strip()
        output = str_in

        if 'county, ca' in str_in:
            str_in = str_in.replace('county, ca', '')
            str_in = str_in.strip()
            output = str_in
            
        if 'county' in str_in:
            str_in = str_in.replace('county', '')
            str_in = str_in.strip()
            output = str_in
            
            
    except:
        output = np.nan
        
    return output


# Data Analysis & Results

Include cells that describe the steps in your data analysis.

In [ ]:
# EDA

### Descriptive Data Analysis(Asthma)

In [3]:
## YOUR CODE HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION
# Asthma

### Descriptive Data Analysis(AQI)

In [ ]:
# AQI


### Descriptive Data Analysis(Cancer)


In [ ]:
# County cancer

In [ ]:
#----------------------------------------------------------------------#
# inferences

In [12]:
county_cancer_2010 = county_cancer_2010.drop(county_cancer_2010.index[39:47])


aqi_cancer_2010 = pd.merge(county_cancer_2010, aqi_2010, on='county')
aqi_cancer_2010.head()

,county,"rate per 100,000",count,Good days,Unhealthy days,AQI 90th Percentile,AQI Median
0,alameda,44.8118,646,245,4,68.0,43.0
1,butte,63.0616,171,208,13,84.0,47.0
2,contra costa,50.2429,536,307,8,64.0,39.0
3,el dorado,54.2364,124,251,23,93.0,42.0
4,fresno,51.7782,401,99,65,130.0,70.0


# Ethics & Privacy

*Fill in your ethics & privacy discussion here*

# Conclusion & Discussion

*Fill in your discussion information here*

# Team Contributions

*Specify who in your group worked on which parts of the project.*